In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras import optimizers
from keras.utils.np_utils import to_categorical
from keras.layers.core import Dropout, Activation, Dense, Reshape, Flatten
from keras.layers import concatenate
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


BATCH_SIZE = 128
BATCHES_PER_EPOCH = 2000
TRAIN_EPOCHS = 40
VAL_EPOCHS = 20

Using TensorFlow backend.


In [2]:
def read_labels():  
    label_file = 'data/batches.meta.txt'
    labels = []
    with open(label_file, 'r') as f:
        for line in f:
            labels.append(line.strip('\n'))
    labels = np.array(labels)
    return labels


def read_data(phase='train'):
    train_set_file = ['data/train/data_batch_{}.bin'.format(i) for i in range(1, 6)]
    val_set_file = 'data/val/test_batch.bin'
    if phase == 'train':
        X = np.array([])
        for tfile in train_set_file:
            with open(tfile, 'rb') as f:
                X = np.append(X, np.fromfile(f, dtype=np.uint8))
        X = X.reshape(-1, 3073)
        y = X[:, 0].reshape(-1, 1).astype(int)
        X = X[:, 1:].reshape((-1, 3, 32, 32)).transpose((0, 2, 3, 1)) / 255
    elif phase == 'val':
        with open(val_set_file, 'rb') as f:
            X = np.fromfile(f, dtype=np.uint8)
        X = X.reshape(-1, 3073)
        y = X[:, 0].reshape(-1, 1).astype(int)
        X = X[:, 1:].reshape((-1, 3, 32, 32)).transpose((0, 2, 3, 1)) / 255
    else:
        raise NotImplementedError
    y = to_categorical(y, num_classes=None)
    return X, y

x_train, y_train = read_data()
x_val, y_val = read_data('val')

In [3]:
def cnn_model(img_rows=32, img_cols=32):                      
    model = Sequential()

    model.add(Conv2D(32, 3, padding='same', activation='relu', input_shape=(img_rows, img_cols, 3)))
    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D())

    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D())
    
    model.add(Conv2D(128, 3, padding='same', activation='relu'))
    model.add(Conv2D(128, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D())

    model.add(Conv2D(256, 3, padding='same', activation='relu'))
    model.add(Conv2D(256, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D())
    
    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(10, activation='softmax'))
    
    for layer in model.layers:
        print('{:20s}'.format(layer.name), layer.input_shape, layer.output_shape)
        
    return model

model = cnn_model()

sgd = optimizers.SGD(lr=0.1, decay=0.0005, momentum=0.9, nesterov=True)
rmsprop = optimizers.RMSprop(lr=1e-4)

model.compile(rmsprop, 'categorical_crossentropy', metrics=['accuracy'])

conv2d_1             (None, 32, 32, 3) (None, 32, 32, 32)
conv2d_2             (None, 32, 32, 32) (None, 32, 32, 32)
max_pooling2d_1      (None, 32, 32, 32) (None, 16, 16, 32)
conv2d_3             (None, 16, 16, 32) (None, 16, 16, 64)
conv2d_4             (None, 16, 16, 64) (None, 16, 16, 64)
max_pooling2d_2      (None, 16, 16, 64) (None, 8, 8, 64)
conv2d_5             (None, 8, 8, 64) (None, 8, 8, 128)
conv2d_6             (None, 8, 8, 128) (None, 8, 8, 128)
max_pooling2d_3      (None, 8, 8, 128) (None, 4, 4, 128)
conv2d_7             (None, 4, 4, 128) (None, 4, 4, 256)
conv2d_8             (None, 4, 4, 256) (None, 4, 4, 256)
max_pooling2d_4      (None, 4, 4, 256) (None, 2, 2, 256)
flatten_1            (None, 2, 2, 256) (None, 1024)
dense_1              (None, 1024) (None, 512)
dropout_1            (None, 512) (None, 512)
dense_2              (None, 512) (None, 512)
dropout_2            (None, 512) (None, 512)
dense_3              (None, 512) (None, 10)


In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest',
    rotation_range=10)
val_datagen = ImageDataGenerator(
    rescale=1./255)

train_generator = train_datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)
val_generator = val_datagen.flow(x_val, y_val, batch_size=BATCH_SIZE)

In [5]:
tb = TensorBoard('data/logs')
cp = ModelCheckpoint('data/weights.{epoch:02d}-{val_loss:.2f}.hdf5')

model.fit_generator(train_generator, BATCHES_PER_EPOCH, TRAIN_EPOCHS,
                    validation_data=val_generator, validation_steps=VAL_EPOCHS,
                    verbose=2, callbacks=[tb, cp], initial_epoch=0)

Epoch 1/40
96s - loss: 2.1089 - acc: 0.2095 - val_loss: 1.8439 - val_acc: 0.3301
Epoch 2/40
95s - loss: 1.8611 - acc: 0.3127 - val_loss: 1.6522 - val_acc: 0.4055
Epoch 3/40
95s - loss: 1.6859 - acc: 0.3839 - val_loss: 1.4558 - val_acc: 0.4758
Epoch 4/40
95s - loss: 1.5551 - acc: 0.4347 - val_loss: 1.3172 - val_acc: 0.5195
Epoch 5/40
95s - loss: 1.4407 - acc: 0.4807 - val_loss: 1.2464 - val_acc: 0.5457
Epoch 6/40
95s - loss: 1.3426 - acc: 0.5194 - val_loss: 1.2154 - val_acc: 0.5478
Epoch 7/40
95s - loss: 1.2545 - acc: 0.5525 - val_loss: 1.0385 - val_acc: 0.6320
Epoch 8/40
95s - loss: 1.1751 - acc: 0.5844 - val_loss: 1.0429 - val_acc: 0.6250
Epoch 9/40
95s - loss: 1.1038 - acc: 0.6116 - val_loss: 0.9575 - val_acc: 0.6708
Epoch 10/40
95s - loss: 1.0359 - acc: 0.6372 - val_loss: 1.0240 - val_acc: 0.6359
Epoch 11/40
95s - loss: 0.9748 - acc: 0.6600 - val_loss: 1.0083 - val_acc: 0.6590
Epoch 12/40
95s - loss: 0.9185 - acc: 0.6800 - val_loss: 0.8346 - val_acc: 0.7051
Epoch 13/40
95s - loss: 0